<a href="https://colab.research.google.com/github/kyunghyuncho/ammi-2019-nlp/blob/master/01-day-LM/ngram_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# N-gram Language Modeling

### Install if needed

TODO: should we install as needed and import as needed or all at once?

In [ ]:
# # run if you dont have it installed
# !pip install more_itertools
# !pip install spacy# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager\
# !python -m spacy download en_core_web_sm

### Imports

In [113]:
import spacy
import torch
from torch.utils.data import Dataset, DataLoader
import random
import numpy
import itertools
from operator import itemgetter 
from glob import glob
from tqdm import tqdm_notebook, tqdm
from collections import Counter
import torch.nn as nn
import torch.nn.functional as F
import string
import re
import more_itertools as mit  # not built-in package
import torch
import torchtext
import torchtext.data as data
from torchtext import vocab
from collections import Counter
import re
from torchtext.data import TabularDataset 

In [114]:
torch.manual_seed(1)


## Data Processing

In [115]:
class AmazonReviewsDataset(TabularDataset):
    
    urls = [
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Books_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Movies_and_TV_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_CDs_and_Vinyl_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Clothing_Shoes_and_Jewelry_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Home_and_Kitchen_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Kindle_Store_5.json.gz',
           'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Sports_and_Outdoors_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Health_and_Personal_Care_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Toys_and_Games_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Video_Games_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Tools_and_Home_Improvement_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Beauty_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Apps_for_Android_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Pet_Supplies_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Automotive_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Grocery_and_Gourmet_Food_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Patio_Lawn_and_Garden_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Baby_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Digital_Music_5.json.gz',
#            'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz',
#             'http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Amazon_Instant_Video_5.json.gz',
        ]
    name='amazonreviews'
    dirname='processed'

In [116]:
# download_done = AmazonReviewsDataset.download(root='data/', check=True)

In [117]:
# RETOK = re.compile(r'\w+|[^\w\s]|\n', re.UNICODE)

# def tokenize(s):
#     return RETOK.findall(s)

# text_field = data.Field(sequential=True, tokenize=tokenize, include_lengths=True, use_vocab=True, lower=True, batch_first=True)

In [118]:
# dataset = AmazonReviewsDataset(path='/home/roberta/ammi-2019-nlp/01-day-LM/data/amazonreviews/reviews_Sports_and_Outdoors_5.json', format='json', fields={'reviewText': ('reviewText', text_field), 'summary': ('summary', text_field)})

In [119]:
# # lets check it
# # lets use fstrings btw
# print(f'Number of samples : {len(dataset.examples)}')

# for ex in dataset.examples:
#     print(f'Review: \n {ex.reviewText} \n\n Summary: \n {ex.summary}')
#     break

In [120]:
# # convert the dataset to a list of strings
# # each string represents a review
# all_reviews = []
# for ex in dataset.examples:
#     all_reviews.append(ex.reviewText)
# len(all_reviews)


In [121]:
# _tqdm = tqdm_notebook  # prolly you need jupyter widget for this, change for tqdm for simple tqdm

# NUM_SENTENCES = len(all_reviews)
# NUM_SENTENCES_TRAIN = int(0.8*NUM_SENTENCES)
# NUM_SENTENCES_TEST = int(0.1*NUM_SENTENCES)
# NUM_SENTENCES_VALID = NUM_SENTENCES - NUM_SENTENCES_TRAIN - NUM_SENTENCES_TEST

# train_reviews = all_reviews[:NUM_SENTENCES_TRAIN]
# test_reviews = all_reviews[NUM_SENTENCES_TRAIN:NUM_SENTENCES_TRAIN+NUM_SENTENCES_TEST]
# valid_reviews = all_reviews[NUM_SENTENCES_TRAIN+NUM_SENTENCES_TEST:NUM_SENTENCES_TRAIN+NUM_SENTENCES_TEST+NUM_SENTENCES_VALID]

# train_reviews = [str(t) for t in train_reviews]
# test_reviews = [str(t) for t in test_reviews]
# valid_reviews = [str(t) for t in valid_reviews]


#### TODO make a function to convert list of urls to txt files with train, test, and valid data

In [122]:
# with open('../data/amazon_reviews_sports_train.txt', 'w') as f:
#     for item in train_reviews:
#         f.write("%s\n" % item)

 

In [123]:
# with open('../data/amazon_reviews_sports_valid.txt', 'w') as f:
#     for item in valid_reviews:
#         f.write("%s\n" % item)
        


In [124]:
# with open('../data/amazon_reviews_sports_test.txt', 'w') as f:
#     for item in test_reviews:
#         f.write("%s\n" % item)

### Load Data From .txt Files

In [125]:
# Load Train Data
train_data = []
with open('../data/amazon_reviews_sports_train.txt', 'r') as f:
    for item in train_reviews:
        train_data.append(item)

In [126]:
# Inspect Train Data        
len(train_data), type(train_data), train_data[0], type(train_data[0]), len(train_data[0])

(237069,
 list,
 "['this', 'came', 'in', 'on', 'time', 'and', 'i', 'am', 'veru', 'happy', 'with', 'it', ',', 'i', 'haved', 'used', 'it', 'already', 'and', 'it', 'makes', 'taking', 'out', 'the', 'pins', 'in', 'my', 'glock', '32', 'very', 'easy']",
 str,
 226)

In [127]:
# Validation Data
valid_data = []
with open('../data/amazon_reviews_sports_valid.txt', 'r') as f:
    for item in valid_reviews:
        valid_data.append(item)

In [128]:
# Inspect Valid Data
len(valid_data), type(valid_data), valid_data[0], type(valid_data[0]), len(valid_data[0])

(29635,
 list,
 "['great', 'sling', 'bag', '.', 'i', 'used', 'it', 'for', 'a', 'trip', 'to', 'disney', 'and', 'this', 'bag', 'worked', 'very', 'well', '.', 'it', 'has', 'enough', 'pockets', 'to', 'carry', 'all', 'the', 'essentials', 'needed', 'for', 'a', 'vacation', 'to', 'disney', 'in', 'june', '.', 'i', 'was', 'able', 'to', 'carry', 'so', 'mush', 'stuff', 'in', 'this', 'bag', 'my', 'wife', 'did', 'not', 'have', 'to', 'carry', 'a', 'back', 'pack', '.', 'i', 'gonna', 'to', 'carry', 'this', 'bag', 'for', 'all', 'my', 'trips', 'from', 'now', 'on', 'and', 'it', 'worked', 'well', 'through', 'all', 'the', 'airport', 'security', 'checks', '.', 'i', 'highly', 'recommend', 'this', 'bag', '.']",
 str,
 676)

In [129]:
# Test Data
test_data = []
with open('../data/amazon_reviews_sports_test.txt', 'r') as f:
    for item in test_reviews:
        test_data.append(item)

In [130]:
# Inspect Test Data
len(test_data), type(test_data), test_data[0], type(test_data[0]), len(test_data[0])

(29633,
 list,
 '[\'this\', \'is\', \'the\', \'most\', \'affordable\', \'/\', \'highest\', \'quality\', \'made\', \'in\', \'the\', \'usa\', \'knife\', \'i\', \'have\', \'found\', \'.\', \'the\', \'knife\', \'is\', \'very\', \'lightweight\', \',\', \'lighter\', \'than\', \'my\', \'kershaw\', \'blur\', \'and\', \'its\', \'a\', \'bit\', \'thinner\', \'too\', \'.\', \'typically\', \'really\', \'lightweight\', \'knives\', \'have\', \'frn\', \'handles\', \'like\', \'the\', \'endura\', \'4\', \'or\', \'the\', \'griptillian\', \'and\', \'that\', "\'", \'s\', \'fine\', \'but\', \'that\', \'stuff\', \'does\', \'flex\', \'if\', \'u\', \'squeeze\', \'hard\', \'enough\', \'.\', \'the\', \'aluminum\', \'handles\', \'in\', \'the\', \'knockout\', \'feel\', \'really\', \'strong\', \'and\', \'have\', \'no\', \'give\', \'to\', \'them\', \'and\', \'i\', \'prefer\', \'the\', \'feel\', \'of\', \'metal\', \'over\', \'plastic\', \'any\', \'day\', \'.\', \'there\', \'was\', \'a\', \'little\', \'rattle\', \'if\

### Process the Data

In [131]:
# Load English tokenizer, tagger, parser, NER and word vectors
tokenizer = spacy.load('en_core_web_sm')               
punctuations = string.punctuation   
TAG_RE = re.compile(r'<[^>]+>') # get rid off HTML tags from the data

def remove_tags(text):
    return TAG_RE.sub('', text)

def lower_case_remove_punc(parsed):
    return [token.text.lower() for token in parsed if (token.text not in punctuations)] #and (token.is_stop is False)]

def tokenize_dataset(dataset):
   # tokenize each sentence -- each tokenized sentence will be an element in token_dataset
    token_dataset = []
    # tokenize all words -- each token will be an item in all_tokens (in the order given by the list of sentences)
    all_tokens = []     # all the tokens -- 

    for sample in tqdm(tokenizer.pipe(dataset, disable=['parser', 'tagger', 'ner'], batch_size=512, n_threads=1)):
        tokens = lower_case_remove_punc(sample)
        token_dataset.append(tokens)
        all_tokens += tokens
        
    return token_dataset, all_tokens

In [132]:
punctuations

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [133]:
TAG_RE

re.compile(r'<[^>]+>', re.UNICODE)

In [134]:
train_data[:2]

["['this', 'came', 'in', 'on', 'time', 'and', 'i', 'am', 'veru', 'happy', 'with', 'it', ',', 'i', 'haved', 'used', 'it', 'already', 'and', 'it', 'makes', 'taking', 'out', 'the', 'pins', 'in', 'my', 'glock', '32', 'very', 'easy']",
 '[\'i\', \'had\', \'a\', \'factory\', \'glock\', \'tool\', \'that\', \'i\', \'was\', \'using\', \'for\', \'my\', \'glock\', \'26\', \',\', \'27\', \',\', \'and\', \'17\', \'.\', \'i\', "\'", \'ve\', \'since\', \'lost\', \'it\', \'and\', \'had\', \'needed\', \'another\', \'.\', \'since\', \'i\', "\'", \'ve\', \'used\', \'ghost\', \'products\', \'prior\', \',\', \'and\', \'know\', \'that\', \'they\', \'are\', \'reliable\', \',\', \'i\', \'had\', \'decided\', \'to\', \'order\', \'this\', \'one\', \'.\', \'sure\', \'enough\', \',\', \'this\', \'is\', \'just\', \'as\', \'good\', \'as\', \'a\', \'factory\', \'tool\', \'.\']']

In [135]:
# TODO: for now only work with small subset of the data -- switch to all data later
train_data = train_data[:800]
test_data = test_data[:100]
valid_data = valid_data[:100]

In [137]:
train_data[0], len(train_data)

("['this', 'came', 'in', 'on', 'time', 'and', 'i', 'am', 'veru', 'happy', 'with', 'it', ',', 'i', 'haved', 'used', 'it', 'already', 'and', 'it', 'makes', 'taking', 'out', 'the', 'pins', 'in', 'my', 'glock', '32', 'very', 'easy']",
 800)

In [139]:
# Tokenize the Datasets
# TODO: this takes a really long time !! why?
train_data_tokenized, all_tokens_train = tokenize_dataset(train_data)
test_data_tokenized, all_tokens_test = tokenize_dataset(test_data)
valid_data_tokenized, all_tokens_valid = tokenize_dataset(valid_data)





0it [00:00, ?it/s]


21it [00:00, 208.14it/s]


45it [00:00, 215.92it/s]


60it [00:00, 189.16it/s]


73it [00:00, 161.85it/s]


90it [00:00, 161.93it/s]


105it [00:00, 155.52it/s]


124it [00:00, 164.10it/s]


147it [00:00, 179.18it/s]


165it [00:00, 170.87it/s]


182it [00:01, 134.80it/s]


197it [00:01, 130.96it/s]


218it [00:01, 144.70it/s]


234it [00:01, 147.27it/s]


255it [00:01, 160.89it/s]


278it [00:01, 176.74it/s]


312it [00:01, 204.32it/s]


337it [00:01, 215.24it/s]


366it [00:01, 232.52it/s]


391it [00:02, 227.78it/s]


415it [00:02, 213.19it/s]


438it [00:02, 193.75it/s]


459it [00:02, 177.22it/s]


478it [00:02, 175.09it/s]


501it [00:02, 186.21it/s]


521it [00:02, 176.84it/s]


540it [00:02, 180.48it/s]


562it [00:03, 181.96it/s]


581it [00:03, 145.23it/s]


601it [00:03, 150.78it/s]


630it [00:03, 175.36it/s]


650it [00:03, 144.82it/s]


673it [00:03, 160.97it/s]


693it [00:03, 170.25it/s]


712it [00:04, 166.09it/s]


730it [00:04, 169.21it/s]


7

Let's look at the tokenized data!

In [150]:
# Number of Tokenized Sentences == Number of All Sentences in the Dataset
# Compare the Tokenized Sentences with the Original Ones
len(train_data_tokenized), train_data_tokenized[:2], train_data[:2]

(800,
 [['this',
   'came',
   'in',
   'on',
   'time',
   'and',
   'i',
   'am',
   'veru',
   'happy',
   'with',
   'it',
   'i',
   'haved',
   'used',
   'it',
   'already',
   'and',
   'it',
   'makes',
   'taking',
   'out',
   'the',
   'pins',
   'in',
   'my',
   'glock',
   '32',
   'very',
   'easy'],
  ['i',
   'had',
   'a',
   'factory',
   'glock',
   'tool',
   'that',
   'i',
   'was',
   'using',
   'for',
   'my',
   'glock',
   '26',
   '27',
   'and',
   '17',
   'i',
   've',
   'since',
   'lost',
   'it',
   'and',
   'had',
   'needed',
   'another',
   'since',
   'i',
   've',
   'used',
   'ghost',
   'products',
   'prior',
   'and',
   'know',
   'that',
   'they',
   'are',
   'reliable',
   'i',
   'had',
   'decided',
   'to',
   'order',
   'this',
   'one',
   'sure',
   'enough',
   'this',
   'is',
   'just',
   'as',
   'good',
   'as',
   'a',
   'factory',
   'tool']],
 ["['this', 'came', 'in', 'on', 'time', 'and', 'i', 'am', 'veru', 'happy',

In [151]:
# Number of All Tokens
len(all_tokens_train), all_tokens_train[:2]

(77653, ['this', 'came'])

#### Build the Vocabulary 


In [153]:
# TODO: do we use both train and valid and not test for this??
voc = list(set(all_tokens_train + all_tokens_valid))
print('Word vocabulary size: {} words'.format(len(voc)))             

Word vocabulary size: 6280 words


### CORPUS ANALYSIS (Train + Valid Data)

#### Number of Tokens in the Corpus Data


In [167]:
print("Number of All Tokens ", len(all_tokens_train) + len(all_tokens_valid))

Number of All Tokens  85062


In [158]:
print("Number of All UNIQUE Tokens ", len(voc))

Number of All UNIQUE Tokens  6280


#### Number of Sentences in the Train Data


In [168]:
print("Number of Sentences ", len(train_data_tokenized), len(valid_data_tokenized))

Number of Sentences  800 100


#### Count how often each sentence length occurs. Visualize this! 

#### Average sentence length

### N-grams

#### Generate a list of words and their corresponding frequencies. Which are the 10 most frequent words?



#### Pad sentences with special markers sentence beginning and end, i.e. $<bos>$ and $<eos>$

In [189]:
def pad_sentences(input_list, n):
    result_list = []
    for l in input_list:
        padded = ["<bos>" for i in range((n - 1))] + l +["<eos>" for i in range((n - 1))]
        result_list.append(padded)
    return result_list

In [191]:
NGRAM = 2  # change this to make different N grams for each token

train_data_padded = pad_sentences(train_data_tokenized, NGRAM)
test_data_padded = pad_sentences(test_data_tokenized, NGRAM)
valid_data_padded = pad_sentences(valid_data_tokenized, NGRAM)


Let's check our padding!

In [194]:
train_data_padded[0]

['<bos>',
 'this',
 'came',
 'in',
 'on',
 'time',
 'and',
 'i',
 'am',
 'veru',
 'happy',
 'with',
 'it',
 'i',
 'haved',
 'used',
 'it',
 'already',
 'and',
 'it',
 'makes',
 'taking',
 'out',
 'the',
 'pins',
 'in',
 'my',
 'glock',
 '32',
 'very',
 'easy',
 '<eos>']

### Function for finding all N-grams

In [195]:
def find_ngrams(input_list, n):
    result_list = []
    for l in input_list:
        result_list.append(list(zip(*[l[i:] for i in range(n)])))
    return result_list

#### Convert the dataset to its corresponding n-gram version

In [202]:
NGRAM = 2  # change this to make different N grams for each token

# now make train and valid dicts
train_data_ngram = find_ngrams(train_data_padded, NGRAM)
valid_data_ngram = find_ngrams(valid_data_padded, NGRAM)
test_data_ngram = find_ngrams(test_data_padded, NGRAM)

Let's check our n-grams!

In [203]:
train_data_ngram[0]

[('<bos>', 'this'),
 ('this', 'came'),
 ('came', 'in'),
 ('in', 'on'),
 ('on', 'time'),
 ('time', 'and'),
 ('and', 'i'),
 ('i', 'am'),
 ('am', 'veru'),
 ('veru', 'happy'),
 ('happy', 'with'),
 ('with', 'it'),
 ('it', 'i'),
 ('i', 'haved'),
 ('haved', 'used'),
 ('used', 'it'),
 ('it', 'already'),
 ('already', 'and'),
 ('and', 'it'),
 ('it', 'makes'),
 ('makes', 'taking'),
 ('taking', 'out'),
 ('out', 'the'),
 ('the', 'pins'),
 ('pins', 'in'),
 ('in', 'my'),
 ('my', 'glock'),
 ('glock', '32'),
 ('32', 'very'),
 ('very', 'easy'),
 ('easy', '<eos>')]

#### Generate a complete list of trigrams occurring in the corpus. Which are the 10 most frequent trigrams?



#### Determine count statistics on the trigram frequencies, i.e. compute so-called count-counts (how many trigrams occur once, twice, : : :). Plot their distribution.

### Create Dictionary of N-grams

In [ ]:
max_vocab_size = 100000

# save index 1 for unk, 0 for pad, 1 for bos, 2 for eos
PAD_IDX = 0
UNK_IDX = 1
BOS_IDX = 2
EOS_IDX = 3

all_train_tokens = list(mit.flatten(train_data_ngram + valid_data_ngram))
counted_tokens = Counter(all_train_tokens)

vocab, count = zip(*counted_tokens.most_common(max_vocab_size))
id2token = list(vocab)
token2id = dict(zip(vocab, range(2,2+len(vocab)))) 
id2token = ['<pad>', '<unk>', '<bos>', '<eos>'] + id2token

token2id['<pad>'] = PAD_IDX 
token2id['<unk>'] = UNK_IDX
token2id['<bos>'] = BOS_IDX 
token2id['<eos>'] = EOS_IDX

In [215]:
# Let's look at some numbers!
len(all_train_tokens), len(vocab), len(count)

(85962, 41951, 41951)

Lets check the dictionary by loading random token from it!


In [209]:
random_token_id = random.randint(0, len(id2token)-1)
random_token = id2token[random_token_id]

print ("Token id {} ; token {}".format(random_token_id, id2token[random_token_id]))
print ("Token {}; token id {}".format(random_token, token2id[random_token]))

Token id 24749 ; token ('belt', 'just')
Token ('belt', 'just'); token id 24747


#### Generate a list of trigrams in the corpus using only the words in the vocabulary. 

#### Generate count statistics of the trigram frequencies for this modified corpus as well. What do you notice in comparison to the previous exercise?

#### Determine the out-of-vocabulary (OOV) rate, i.e. the percentage of running words in the corpus which are not covered by the vocabulary.

### Functions for Converting from Token to ID and back

In [221]:
def _text2id(doc):
    return [token2id[t] if t in token2id else UNK_IDX for t in doc]

def _id2text(vec):
    return [id2token[i] for i in vec]
    

In [229]:
train_data_id = []
for d in train_data_ngram:
    train_data_id.append(_text2id(d))
    
valid_data_id = []
for d in valid_data_ngram:
    valid_data_id.append(_text2id(d))
    
train_data_id_merged = []
for d in train_data_id:
    train_data_id_merged.append((d, 0))

valid_data_id_merged = []
for d in valid_data_id:
    valid_data_id_merged.append((d, 0))


In [242]:
# let's look at some numbers
train_data_id[0][:2], train_data_ngram[0][:2], train_data_id_merged[0], len(train_data_id_merged), len(train_data_id)

([21, 9863],
 [('<bos>', 'this'), ('this', 'came')],
 ([21,
   9863,
   2505,
   3415,
   3416,
   367,
   16,
   49,
   9864,
   9865,
   396,
   133,
   136,
   9866,
   9867,
   432,
   5237,
   9868,
   18,
   884,
   9869,
   9870,
   321,
   9871,
   9872,
   28,
   1313,
   9873,
   9874,
   699,
   9875],
  0),
 800,
 800)

## Making PyTorch Dataset out of our set of dicts

In [238]:
class ImdbDataset(Dataset):
    def __init__(self, data_list, max_inp_length=None, device='cpu'):
        """
        data_list is a list of tuples: (x,y) where x is a list of ids and y is a label
        """
        self.data = data_list
        self.max_len = max_inp_length
        self.data_tensors = []
        for (i, t) in tqdm_notebook(self.data):
            self.data_tensors.append((torch.LongTensor(i[:self.max_len]), torch.LongTensor([t])))
            #TODO: fix error regarding to(device)
#             self.data_tensors.append((torch.LongTensor(i[:self.max_len]).to(device), torch.LongTensor([t]).to(device)))
              
    def __getitem__(self, key):
        (inp, tgt) = self.data_tensors[key]
        
        return inp, tgt, len(inp)

    def __len__(self):
        return len(self.data)

def pad(tensor, length, dim=0, pad=0):
    """Pad tensor to a specific length.
    :param tensor: vector to pad
    :param length: new length
    :param dim: (default 0) dimension to pad
    :returns: padded tensor if the tensor is shorter than length
    """
    if tensor.size(dim) < length:
        return torch.cat(
            [tensor, tensor.new(*tensor.size()[:dim],
                                length - tensor.size(dim),
                                *tensor.size()[dim + 1:]).fill_(pad)],
            dim=dim)
    else:
        return tensor
    
def batchify(batch):
    maxlen = max(batch, key = itemgetter(2))[-1]
    batch_list = []
    target_list = []
    for b in batch:
        batch_list.append(pad(b[0], maxlen, dim=0, pad=PAD_IDX))
        target_list.append(b[1])
    input_batch = torch.stack(batch_list, 0)
    target_batch = torch.stack(target_list, 0)
    
    return input_batch, target_batch

In [106]:
train_dataset = ImdbDataset(train_data_id_merged, max_inp_length=None, device='cuda')
train_loader = DataLoader(train_dataset, batch_size=512, collate_fn=batchify, shuffle=True)

valid_dataset = ImdbDataset(valid_data_id_merged, max_inp_length=None, device='cuda')
valid_loader = DataLoader(valid_dataset, batch_size=512, collate_fn=batchify, shuffle=True)

In [16]:
out = vocab.FastText(language='en')

.vector_cache/wiki.en.vec:  19%|█▉        | 1.25G/6.60G [11:49<50:35, 1.76MB/s]


KeyboardInterrupt: 

In [ ]:
text_field.build_vocab(dataset, max_size=30000, vectors=out)

In [ ]:
# making a batch iterator
train_loader = data.BucketIterator(dataset=dataset, batch_size=4, sort_key=lambda x: len(x.reviewText), device=torch.device('cpu'), sort_within_batch=True, repeat=False)

In [ ]:
batch = next(iter(train_loader))
print(batch)

In [ ]:
def _vec2txt(vec):
    return [text_field.vocab.itos[t] for t in vec]

In [ ]:
print(batch.reviewText[0][0])
print(_vec2txt(batch.reviewText[0][0]))

# **Language Modeling**

**Goal:** compute a probabilty distribution over all possible sentences:


$p(W) = p(w_1, w_2, ..., w_T)$ - the probability that the sentence composed of words $(w_1, ..., w_T)$ in this order




This unsupervised learning problem can be framed as a sequence of supervised learning problems:


$p(W) = p(w_1) * p(w_2|w_1) * ... * p(w_T|w_1, ..., w_{T-1})$



If we have N sentences, each of them with T words / tokens, then we want to max:

$log p(W) = \sum_{n = 1}^N \sum_{i=1}^{T} log p(w_i | w_{<i})$




# **N-gram language model**

**Goal:** estimate the n-gram probabilities using counts of sequences of n consecutive words

Given a sequence of words $w$, we want to compute

  $P(w_i|w_{i−1}, w_{i−2}, …, w_{i−n+1})$

Where $w_i$ is the i-th word of the sequence.

$P(w_i|w_{i−n+1}, ..., w_{i−2}, w_{i−1}) = \frac{p(w_{i−n+1}, ..., w_{i−2}, w_{i−1}, w_i)}{\sum_{w \in V} p(w_{i−n+1}, ..., w_{i−2}, w_{i−1}, w)}$

**Key Idea:** We can estimate the probabilities using counts of n-grams in our dataset 


# Data Processing

# Bag of N-grams

# Train, Validation, Test Datasets

# Model

# Loss Function and Optimizer

# Training and Testing

# Analysis & Examples